### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import pickle 

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t' , header=None)
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

### Split into train/test

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data[['body_text', 'body_len', 'punct%']], data['label'], test_size=0.2 , random_state=42 , shuffle=True , stratify=data['label'])

### Vectorize text

In [3]:
# since the indices do not match we need to drop the older index
# but the text messages are in the same order
X_train[['body_len' , 'punct%']].head()

,body_len,punct%
1835,132,4.5
4131,44,4.5
2873,41,4.9
967,69,5.8
3164,123,8.9


In [4]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

# so the validation set will contain only the features seen on the training set
tfidf_train = tfidf_vect.transform(X_train['body_text'])
tfidf_val = tfidf_vect.transform(X_val['body_text'])

X_train_vect = pd.concat([X_train[['body_len' , 'punct%']].reset_index(drop=True) , 
           pd.DataFrame(tfidf_train.toarray() , columns=tfidf_vect.get_feature_names_out())] , axis=1)

X_val_vect = pd.concat([X_val[['body_len' , 'punct%']].reset_index(drop=True) , 
           pd.DataFrame(tfidf_val.toarray() , columns=tfidf_vect.get_feature_names_out())] , axis=1)

X_train_vect.head()

,body_len,punct%,,0,008704050406,0089mi,0121,01223585236,01223585334,0125698789,...,zero,zhong,zindgi,zoe,zogtoriu,zouk,zyada,ü,üll,〨ud
0,132,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,44,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,41,4.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,69,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,123,8.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Final evaluation of models

In [5]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [6]:
rf = RandomForestClassifier(n_estimators=150, max_depth=None, random_state=42,n_jobs=-1)

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = rf_model.predict(X_val_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_val, y_pred, pos_label='spam', average='binary')
print(f'Fit time : {round(fit_time,3)} / Pred time : {round(pred_time,3)} / Precision: {round(precision, 3)} / Recall: {round(recall, 3)} / Accuracy: {round((y_pred==y_val).sum()/len(y_pred), 3)}')

Fit time : 1.962 / Pred time : 0.127 / Precision: 1.0 / Recall: 0.799 / Accuracy: 0.973


In [7]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)
start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = gb_model.predict(X_val_vect)
end = time.time()
pred_time = (end - start)


precision, recall, fscore, train_support = score(y_val, y_pred, pos_label='spam', average='binary')
print(f'Fit time : {round(fit_time,3)} / Pred time : {round(pred_time,3)} / Precision: {round(precision, 3)} / Recall: {round(recall, 3)} / Accuracy: {round((y_pred==y_val).sum()/len(y_pred), 3)}')

Fit time : 129.432 / Pred time : 0.222 / Precision: 0.907 / Recall: 0.852 / Accuracy: 0.969


In [8]:
# Save the trained model and vectorizer
with open('spam_classifier_model.pkl', 'wb') as model_file:
    pickle.dump(gb_model, model_file)
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf_vect, vectorizer_file)